In [1]:
import findspark
findspark.init('/home/osboxes/spark-2.4.3-bin-hadoop2.7')
import pyspark
import os

In [4]:
myPath = os.path.join('/home', 'osboxes', 'CourseMaterial', 'Spark_for_Machine_Learning', 'Clustering')

print(myPath)

os.chdir(myPath)

/home/osboxes/CourseMaterial/Spark_for_Machine_Learning/Clustering


In [5]:
os.listdir()

['Clustering_Consulting_Project.ipynb',
 'Clustering Code Along.ipynb',
 'seeds_dataset.csv',
 'seeds_dataset.txt',
 '.ipynb_checkpoints',
 'Clustering_Code_Example.ipynb',
 'Clustering_Consulting_Project_SOLUTIONS.ipynb',
 'hack_data.csv',
 'sample_kmeans_data.txt']

In [6]:
# set up the session info

from pyspark.sql import SparkSession

name = 'cluster'

spark = SparkSession.builder.appName(name).getOrCreate()

from pyspark.ml.clustering import KMeans

In [7]:
dataset = spark.read.format('libsvm').load('sample_kmeans_data.txt')
dataset.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|           (3,[],[])|
|  1.0|(3,[0,1,2],[0.1,0...|
|  2.0|(3,[0,1,2],[0.2,0...|
|  3.0|(3,[0,1,2],[9.0,9...|
|  4.0|(3,[0,1,2],[9.1,9...|
|  5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+



In [8]:
final_data = dataset.select('features')
final_data.show()

+--------------------+
|            features|
+--------------------+
|           (3,[],[])|
|(3,[0,1,2],[0.1,0...|
|(3,[0,1,2],[0.2,0...|
|(3,[0,1,2],[9.0,9...|
|(3,[0,1,2],[9.1,9...|
|(3,[0,1,2],[9.2,9...|
+--------------------+



In [9]:
kmeans = KMeans().setK(2).setSeed(1)

In [10]:
model = kmeans.fit(final_data)

In [12]:
# within set sum of squares for the inflection point

wssse = model.computeCost(final_data)
print(wssse)

0.11999999999994547


In [14]:
centers = model.clusterCenters()
centers

[array([0.1, 0.1, 0.1]), array([9.1, 9.1, 9.1])]

In [17]:
# find the label

results = model.transform(final_data)
results.show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         0|
|(3,[0,1,2],[0.1,0...|         0|
|(3,[0,1,2],[0.2,0...|         0|
|(3,[0,1,2],[9.0,9...|         1|
|(3,[0,1,2],[9.1,9...|         1|
|(3,[0,1,2],[9.2,9...|         1|
+--------------------+----------+



In [18]:
# reset the centers to 3

kmeans = KMeans().setK(3).setSeed(1)
model = kmeans.fit(final_data)
wssse = model.computeCost(final_data)
print(wssse)

0.07499999999994544


In [21]:
centers = model.clusterCenters()
centers

[array([9.1, 9.1, 9.1]), array([0.05, 0.05, 0.05]), array([0.2, 0.2, 0.2])]

In [22]:
results = model.transform(final_data)
results.show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         1|
|(3,[0,1,2],[0.1,0...|         1|
|(3,[0,1,2],[0.2,0...|         2|
|(3,[0,1,2],[9.0,9...|         0|
|(3,[0,1,2],[9.1,9...|         0|
|(3,[0,1,2],[9.2,9...|         0|
+--------------------+----------+

